In [2]:
KEY = '####################################' #(API Key)

In [5]:
import apiclient
import pandas as pd

youtube = apiclient.discovery.build('Youtube', 'v3', developerKey=KEY)
playlist_id = 'PLvahqwMqN4M2N01FfQy2wXkyVyucAL86b' 
# Initialize variables for pagination
next_page_token = None
video_ids = []

while True:
    # Make a request to the API to retrieve video IDs from the playlist
    results = youtube.playlistItems().list(
        part="contentDetails",
        playlistId=playlist_id,
        maxResults=50,  # Maximum is 50
        pageToken=next_page_token
    ).execute()

    # Extract video IDs and append them to the video_ids list
    for item in results['items']:
        video_ids.append(item['contentDetails']['videoId'])

    # Check if there are more pages
    next_page_token = results.get('nextPageToken')

    # If there are no more pages, break out of the loop
    if not next_page_token:
        break

In [76]:
# insert video ids into a dataframe
df = pd.DataFrame({'video_id': video_ids})

# scrape video details: title, description, tags, category, duration, view_count, 
# like_count, dislike_count, comment_count, published_at based on video ids
video_id = []
title = []
description = []
tags = []
category = []
duration = []
view_count = []
like_count = []
comment_count = []
published_at = []

for i in range(len(video_ids)):
    videoID = video_ids[i]

    # make API call
    response = youtube.videos().list(
        part='snippet, contentDetails, statistics',
        id=videoID
    ).execute()

    # extract video details
    # keep the video id
    video_id.append(videoID)
    title.append(response['items'][0]['snippet']['title'])
    description.append(response['items'][0]['snippet']['description'])
    tags.append(response['items'][0]['snippet']['tags'])
    category.append(response['items'][0]['snippet']['categoryId'])
    duration.append(response['items'][0]['contentDetails']['duration'])
    view_count.append(response['items'][0]['statistics']['viewCount'])
    like_count.append(response['items'][0]['statistics']['likeCount'])
    comment_count.append(response['items'][0]['statistics']['commentCount'])
    published_at.append(response['items'][0]['snippet']['publishedAt'])
    
    
    # import to dataframe
    videos = pd.DataFrame({'video_id': video_id,
                       'title': title,
                       'description': description,
                       #'tags': tags,
                       'category': category,
                       'duration': duration,
                       'view_count': view_count,
                       'like_count': like_count,
                       'comment_count': comment_count,
                       'published_at': published_at
                       })

In [77]:
videos

,video_id,title,description,category,duration,view_count,like_count,comment_count,published_at
0,PrgxJ1_sUcs,Black Clover: Sword of the Wizard King | Offic...,"The Netflix Film ""Black Clover: Sword of the W...",24,PT1M16S,2125478,45845,2247,2023-04-07T04:00:03Z
1,A4JCfv4NuUw,Indian Matchmaking: Season 3 | Official Traile...,"From London to New Delhi, matchmaker Sima Tapa...",24,PT1M49S,226389,1733,251,2023-04-06T15:30:00Z
2,bNLiUEnPMaw,Michelle Obama & Oprah Winfrey | The Light We ...,On the final tour stop for her 2022 bestseller...,24,PT1M48S,181143,2434,483,2023-04-06T13:00:00Z
3,lV6sJlBbhPs,The Diplomat | Official Trailer | Netflix,"In the midst of an international crisis, a car...",24,PT2M15S,2000956,16896,993,2023-04-05T14:00:02Z


In [78]:
# export to csv 
videos.to_csv('video_detail.csv', index=False)